# A simple Sudoku solver in Python

Writing a [Sudoku](https://en.wikipedia.org/wiki/Sudoku) puzzle solver is a nice little programming exercise. A valid puzzle is a 9x9 grid of cells consisting of nine 3x3-cell non-overlapping boxes, and the aim is to place a digit on each cell so that each digits appears only once on each row, only once on each column, and only once on each block. In the beginning, some of the digit placements are given so that the solution of the puzzle is unique.

## Drawing a sudoku

Let's start by defining a small class to hold a Sudoku puzzle.

In [1]:
class Sudoku(object):
    
    def __init__(self, sudoku_as_string, check_valid_characters=True):
        """Initializes the sudoku from a 81-character string containing one character
        for each cell, from left to right, top to bottom."""
        if len(sudoku_as_string) != 81:
            raise ValueError("A Sudoku puzzle must be initialized with an 81-character string!")
        # The next check is optional so that we can abuse the class a bit later
        if check_valid_characters:
            if any(c not in " 123456789" for c in sudoku_as_string):
                raise ValueError("A Sudoku can only contain digits 1-9 and spaces!")
        self.puzzle = sudoku_as_string
        
        
    def _repr_html_(self):
        """Returns a styled HTML representation of the Sudoku puzzle for Jupyter to render."""
        
        # Override the zebra striping of tables in the default Jupyter theme and draw a grid
        # in CSS
        puzzle_css = """
            tr.sudoku-tr {background: #ffffff;}
            td.sudoku-td {border: 1px solid #888888; width: 25pt;
                height: 25pt; text-align: center; font-size: 12pt;}
            tr.sudoku-tr:nth-of-type(3n) {border-bottom: 2px solid black;}
            tr.sudoku-tr:nth-of-type(1) {border-top: 2px solid black;}
            td.sudoku-td:nth-of-type(3n) {border-right: 2px solid black;}
            td.sudoku-td:nth-of-type(1) {border-left: 2px solid black;}
            """
        
        # Construct the puzzle in HTML
        rows = [self.puzzle[9*i:9*(i+1)] for i in range(9)]
        puzzle_table_rows = ["".join(f"<td class='sudoku-td'>{x}</td>" for x in line)
                             for line in rows]
        puzzle_html = ("<style type='text/css'>" + puzzle_css + "</style>" +
                       "<table class='sudoku'>" +
                       "".join(f"<tr class='sudoku-tr'>{line}</tr>" for line in puzzle_table_rows) +
                       "</table>")
        return puzzle_html

In [2]:
# Not really a valid Sudoku, just checking that the Jupyter pretty-printing works as expected
Sudoku("   333   "
       "1 2 9 888"
       " 5     5 "
       "   333   "
       "   333   "
       "   333   "
       "1 2 9 888"
       "1 2 9 888"
       "1 2 9 888")

,,,3,3,3,,,
1,,2,,9,,8,8,8
,5,,,,,,5,
,,,3,3,3,,,
,,,3,3,3,,,
,,,3,3,3,,,
1,,2,,9,,8,8,8
1,,2,,9,,8,8,8
1,,2,,9,,8,8,8


## Solution strategy

The simplest algorithm that could work for this problem would be to try brute-forcing through every possible combination.

Let's try a recursive approach: Given an unsolved puzzle, check first if it's all solved. If it is, return the puzzle, otherwise pick the first empty cell and try each digit on that cell. If a solution was found, return it; if none of the digits yields a solution for the puzzle, it's impossible, so return None.

This way we are implementing a depth-first brute-force search, terminating each branch as soon as it's clear there is no solution that way.

First, let's codify the rules in a way that checking whether a puzzle is valid is easy. I chose to store the puzzle as a simple 81-character string, so string indices 0-80 correspond to cells in a puzzle like this:

In [3]:
Sudoku([str(i) for i in range(81)], False) # Here we abuse the class written above a little

0,1,2,3,4,5,6,7,8
9,10,11,12,13,14,15,16,17
18,19,20,21,22,23,24,25,26
27,28,29,30,31,32,33,34,35
36,37,38,39,40,41,42,43,44
45,46,47,48,49,50,51,52,53
54,55,56,57,58,59,60,61,62
63,64,65,66,67,68,69,70,71
72,73,74,75,76,77,78,79,80


In [4]:
def create_sudoku_groups():
    """Constructs the list of groups of cells in a Sudoku that must contain unique digits.
    Each group of cells is represented by a list of cell numbers (0-80, starting from the upper
    left corner, numbered left to right, top to bottom)."""
    groups = [[icol + 9*irow for icol in range(9)] for irow in range(9)] # rows
    groups += [[icol + 9*irow for irow in range(9)] for icol in range(9)] # columns
    groups += [[3*icol + 27*irow + i for i in [0, 1, 2, 9, 10, 11, 18, 19, 20]]
               for icol in range(3) for irow in range(3)] # blocks
    return groups

sudoku_groups = create_sudoku_groups()
sudoku_groups

[[0, 1, 2, 3, 4, 5, 6, 7, 8],
 [9, 10, 11, 12, 13, 14, 15, 16, 17],
 [18, 19, 20, 21, 22, 23, 24, 25, 26],
 [27, 28, 29, 30, 31, 32, 33, 34, 35],
 [36, 37, 38, 39, 40, 41, 42, 43, 44],
 [45, 46, 47, 48, 49, 50, 51, 52, 53],
 [54, 55, 56, 57, 58, 59, 60, 61, 62],
 [63, 64, 65, 66, 67, 68, 69, 70, 71],
 [72, 73, 74, 75, 76, 77, 78, 79, 80],
 [0, 9, 18, 27, 36, 45, 54, 63, 72],
 [1, 10, 19, 28, 37, 46, 55, 64, 73],
 [2, 11, 20, 29, 38, 47, 56, 65, 74],
 [3, 12, 21, 30, 39, 48, 57, 66, 75],
 [4, 13, 22, 31, 40, 49, 58, 67, 76],
 [5, 14, 23, 32, 41, 50, 59, 68, 77],
 [6, 15, 24, 33, 42, 51, 60, 69, 78],
 [7, 16, 25, 34, 43, 52, 61, 70, 79],
 [8, 17, 26, 35, 44, 53, 62, 71, 80],
 [0, 1, 2, 9, 10, 11, 18, 19, 20],
 [27, 28, 29, 36, 37, 38, 45, 46, 47],
 [54, 55, 56, 63, 64, 65, 72, 73, 74],
 [3, 4, 5, 12, 13, 14, 21, 22, 23],
 [30, 31, 32, 39, 40, 41, 48, 49, 50],
 [57, 58, 59, 66, 67, 68, 75, 76, 77],
 [6, 7, 8, 15, 16, 17, 24, 25, 26],
 [33, 34, 35, 42, 43, 44, 51, 52, 53],
 [60, 61, 62, 69

In [5]:
def solve_sudoku(puzzle, sudoku_groups):
    """Recursively solves the Sudoku. Returns the solution or None, if there is no solution."""
    # First check the validity of the puzzle
    for cell_list in sudoku_groups:
        block_digits = [puzzle.puzzle[i] for i in cell_list if puzzle.puzzle[i] != " "]
        if len(block_digits) != len(set(block_digits)):
            return None
    
    # Find the first empty cell. If no empty cells are found, the puzzle is solved.
    first_empty_cell = puzzle.puzzle.find(" ")
    if first_empty_cell == -1:
        return puzzle
    
    # Try every option on the empty cell. If none works, this is a dead end, return None.
    for digit in "123456789":
        trial = Sudoku(puzzle.puzzle.replace(" ", digit, 1))
        solution = solve_sudoku(trial, sudoku_groups)
        if solution:
            return solution # propagates up when found
    
    return None

Let's try an invalid puzzle first, this should return None:

In [6]:
invalid_puzzle = Sudoku(9*"123 4 421")
solve_sudoku(invalid_puzzle, sudoku_groups) == None

True

## Testing the solver

Let's test it with a sudoku found on the Wikipedia page:

In [7]:
sudoku = Sudoku("53  7    "
                "6  195   "
                " 98    6 "
                "8   6   3"
                "4  8 3  1"
                "7   2   6"
                " 6    28 "
                "   419  5"
                "    8  79")
sudoku

5,3,,,7,,,,
6,,,1,9,5,,,
,9,8,,,,,6,
8,,,,6,,,,3
4,,,8,,3,,,1
7,,,,2,,,,6
,6,,,,,2,8,
,,,4,1,9,,,5
,,,,8,,,7,9


In [8]:
solve_sudoku(sudoku, sudoku_groups)

5,3,4,6,7,8,9,1,2
6,7,2,1,9,5,3,4,8
1,9,8,3,4,2,5,6,7
8,5,9,7,6,1,4,2,3
4,2,6,8,5,3,7,9,1
7,1,3,9,2,4,8,5,6
9,6,1,5,3,7,2,8,4
2,8,7,4,1,9,6,3,5
3,4,5,2,8,6,1,7,9


Eyeballing a few randomly picked rows, columns, and blocks, this seems to indeed be a valid solution, and the solution was quick.

Let's try a harder one and see if our naive approach is too slow then. In fact, let's try [the world's hardest Sudoku by Prof. Arto Inkala](http://www.telegraph.co.uk/news/science/science-news/9359579/Worlds-hardest-sudoku-can-you-crack-it.html):

In [9]:
hard_sudoku = Sudoku("8        "
                     "  36     "
                     " 7  9 2  "
                     " 5   7   "
                     "    457  "
                     "   1   3 "
                     "  1    68"
                     "  85   1 "
                     " 9    4  ")
hard_sudoku

8,,,,,,,,
,,3,6,,,,,
,7,,,9,,2,,
,5,,,,7,,,
,,,,4,5,7,,
,,,1,,,,3,
,,1,,,,,6,8
,,8,5,,,,1,
,9,,,,,4,,


In [10]:
%time solve_sudoku(hard_sudoku, sudoku_groups)

CPU times: user 9.95 s, sys: 10.1 ms, total: 9.96 s
Wall time: 9.97 s


8,1,2,7,5,3,6,4,9
9,4,3,6,8,2,1,7,5
6,7,5,4,9,1,2,8,3
1,5,4,2,3,7,8,9,6
3,6,9,8,4,5,7,2,1
2,8,7,1,6,9,5,3,4
5,2,1,9,7,4,3,6,8
4,3,8,5,2,6,9,1,7
7,9,6,3,1,8,4,5,2


On my laptop, this took about ten seconds. Clearly the simple solver here is good enough to solve any Sudoku thrown its way.

If there are multiple solutions - which there should not be in published Sudoku puzzles - our algorithm returns the first one it finds. Let's see what solution it returns for an empty grid:

In [11]:
solve_sudoku(Sudoku(81*" "), sudoku_groups)

1,2,3,4,5,6,7,8,9
4,5,6,7,8,9,1,2,3
7,8,9,1,2,3,4,5,6
2,1,4,3,6,5,8,9,7
3,6,5,8,9,7,2,1,4
8,9,7,2,1,4,3,6,5
5,3,1,6,4,2,9,7,8
6,4,2,9,7,8,5,3,1
9,7,8,5,3,1,6,4,2
